## Irrelevants

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
import tensorflow as tf

# Check if GPU is available
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs are available:")
    for gpu in gpus:
        print(f"  - {gpu}")
else:
    print("No GPUs found. Using CPU.")

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Constants
IMG_SIZE = 64  # Resize frames to 64x64
FRAMES = 30  # Fixed number of frames per video
CHANNELS = 3  # RGB channels
CLASSES = ['Suspicious', 'Non-suspicious']

# Function to load videos and convert them to arrays
def load_videos_from_folder(folder_path, class_labels):
    data, labels = [], []
    
    for class_name in class_labels:
        class_path = os.path.join(folder_path, class_name)
        
        for video_name in os.listdir(class_path):
            video_path = os.path.join(class_path, video_name)
            frames = extract_video_frames(video_path, FRAMES, IMG_SIZE)
            
            if frames is not None:
                data.append(frames)
                labels.append(class_name)
    
    return np.array(data), np.array(labels)

# Function to extract fixed number of frames from a video
def extract_video_frames(video_path, num_frames, img_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)  # Select evenly spaced frames
    
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        
        if i in frame_indices:
            frame = cv2.resize(frame, (img_size, img_size))
            frames.append(frame)
    
    cap.release()
    
    if len(frames) < num_frames:  # Pad if not enough frames
        frames.extend([frames[-1]] * (num_frames - len(frames)))
    
    return np.array(frames) if len(frames) == num_frames else None

# Load datasets
train_videos, train_labels = load_videos_from_folder("dataset/Train", CLASSES)
val_videos, val_labels = load_videos_from_folder("dataset/Val", CLASSES)
test_videos, test_labels = load_videos_from_folder("dataset/Test", CLASSES)

# Normalize data
train_videos, val_videos, test_videos = train_videos / 255.0, val_videos / 255.0, test_videos / 255.0

# Encode labels
encoder = LabelEncoder()
train_labels_enc = to_categorical(encoder.fit_transform(train_labels))
val_labels_enc = to_categorical(encoder.transform(val_labels))
test_labels_enc = to_categorical(encoder.transform(test_labels))

print(f"Train shape: {train_videos.shape}, Train labels: {train_labels_enc.shape}")


## Extract frames to npy

In [ ]:
import os
import numpy as np
import torch
import torchvision.io as io
import torchvision.transforms as T

# Removes the dataset.npy folder
!rm -rf dataset.npy

IMG_SIZE = 216
FRAMES = 20
CHANNELS = 3
CLASSES = ['Suspicious', 'Non-suspicious']
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Batch resize transform
resize_transform = T.Resize((IMG_SIZE, IMG_SIZE))

def extract_video_frames(video_path, num_frames, img_size):
    try:
        video, _, _ = io.read_video(video_path, pts_unit='sec')
    except Exception as e:
        print(f"Error reading {video_path}: {e}")
        return None

    total_frames = video.shape[0]
    if total_frames < 1:
        return None

    indices = torch.linspace(0, total_frames - 1, num_frames).long()
    frames = video[indices]  # (T, H, W, C)
    frames = frames.permute(0, 3, 1, 2).float() / 255.0  # (T, C, H, W)
    frames = frames.to(DEVICE)

    # Efficient batch resize
    resized = T.functional.resize(frames, [img_size, img_size])  # batch resize
    resized = resized.permute(0, 2, 3, 1).cpu().numpy()  # back to (T, H, W, C)

    return resized

def save_preprocessed_videos(input_folder, output_folder, class_labels):
    os.makedirs(output_folder, exist_ok=True)

    for class_name in class_labels:
        class_input_path = os.path.join(input_folder, class_name)
        class_output_path = os.path.join(output_folder, class_name)
        os.makedirs(class_output_path, exist_ok=True)

        for video_name in os.listdir(class_input_path):
            if not video_name.lower().endswith((".mp4", ".avi", ".mov")):
                continue

            video_path = os.path.join(class_input_path, video_name)
            output_file_path = os.path.join(class_output_path, video_name.replace('.mp4', '.npy'))

            if os.path.exists(output_file_path):
                print(f"Skipping {output_file_path}, already exists.")
                continue

            frames = extract_video_frames(video_path, FRAMES, IMG_SIZE)

            if frames is not None:
                np.save(output_file_path, frames)
                # print(f"Saved: {output_file_path}")
            else:
                print(f"Failed: {video_path}")

save_preprocessed_videos("dataset/Train", "dataset.npy/Train", CLASSES)
save_preprocessed_videos("dataset/Val", "dataset.npy/Val", CLASSES)
save_preprocessed_videos("dataset/Test", "dataset.npy/Test", CLASSES)

print("Preprocessing complete.")